### Test de detecciond de poses con mediapipe


In [1]:
## Importaciones
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import numpy as np
import time
import csv

In [2]:
## Modelo
model_path = "Models/pose_landmarker_full.task"

In [3]:
## Importaciones de mediapipe
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
PoseLandmarkerResult = mp.tasks.vision.PoseLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [4]:
# globales
latest_frame = None
processing = False


In [5]:
# csv 
csv_file = open('pose_landmarks.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['timestamp', 'landmark_id', 'x', 'y', 'z', 'visibility'])


40

In [6]:


# dibuja las lineas
def draw_landmarks_manually(image, landmarks, connections):
    height, width, _ = image.shape
    for landmark in landmarks:
        x = int(landmark.x * width)
        y = int(landmark.y * height)
        cv2.circle(image, (x, y), 5, (0, 255, 0), -1)
    
    for connection in connections:
        start_idx = connection[0]
        end_idx = connection[1]
        start_point = landmarks[start_idx]
        end_point = landmarks[end_idx]
        
        start_x = int(start_point.x * width)
        start_y = int(start_point.y * height)
        end_x = int(end_point.x * width)
        end_y = int(end_point.y * height)
        
        cv2.line(image, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)


In [7]:

def print_result(result: PoseLandmarkerResult, output_image: mp.Image, timestamp_ms: int): # type: ignore
    global latest_frame, processing
    try:
        frame = output_image.numpy_view()
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if result.pose_landmarks:
            # Draw the pose landmarks
            for landmarks in result.pose_landmarks:
                draw_landmarks_manually(frame, landmarks, mp_pose.POSE_CONNECTIONS)

                for idx, landmark in enumerate(landmarks):
                    # Write landmark data to CSV
                    csv_writer.writerow([timestamp_ms, idx, landmark.x, landmark.y, landmark.z, landmark.visibility])

        latest_frame = frame
    except Exception as e:
        print(f"Error in print_result: {e}")
        import traceback
        traceback.print_exc()
    finally:
        processing = False


In [8]:
options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result)

In [9]:



# crea el modelo
with PoseLandmarker.create_from_options(options) as landmarker:
    # captura el video
    cap = cv2.VideoCapture(0)
    
    # seteo de la camara
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    cap.set(cv2.CAP_PROP_FPS, 30)  
    
    # tiempo de cada frame
    frame_time = 1/30  
    last_time = time.time()

    try:
        while cap.isOpened():
            current_time = time.time()
            delta_time = current_time - last_time

            if delta_time >= frame_time:
                success, frame = cap.read()
                if not success:
                    print("Failed to read frame from webcam")
                    break

                if not processing:
                    processing = True
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)
                    landmarker.detect_async(mp_image, int(current_time * 1000))

                if latest_frame is not None:
                    cv2.imshow('Pose Landmarker', latest_frame)
                else:
                    cv2.imshow('Pose Landmarker', frame)

                last_time = current_time

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        # libera la camara
        cap.release()
        # cierra la ventana
        cv2.destroyAllWindows()
        # cierra el csv
        csv_file.close()  

# imprime la version de opencv y mediapipe  
print(f"OpenCV version: {cv2.__version__}")
print(f"MediaPipe version: {mp.__version__}")

I0000 00:00:1726968711.081439 3155859 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1726968711.188461 3156018 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726968711.206332 3156018 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726968713.143249 3156023 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
/Users/jjmr/PycharmProjects/TestDetection/venv/lib/python3.9/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() w

OpenCV version: 4.10.0
MediaPipe version: 0.10.15


: 